In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
import os
pd.set_option("max_columns", 10000)

%pylab inline
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

import json
from IPython import display

from tqdm import tqdm, tqdm_notebook
from Scripts.saving import submit

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

Populating the interactive namespace from numpy and matplotlib


In [2]:
base_dir = ".."
input_dir = open( os.path.join(base_dir, "datadir.txt"), "r").read()[:-1]
pred_dir = os.path.join(base_dir, "predictions")
processed_dir = os.path.join(input_dir, "processed")
!tree {input_dir}

/home/furfa/work/ai-academy2019/input
├── ai-academy-2019-skill-prediction-data-csv-v1.zip
├── ai-academy-2019-skill-prediction-data-jsonlines.zip
├── dota2_abilities.csv
├── dota2_heroes.csv
├── dota2_items.csv
├── dota2_skill_test.csv
├── dota2_skill_test.jsonlines
├── dota2_skill_train.csv
├── dota2_skill_train.jsonlines
└── processed
    ├── test_proc.csv
    ├── test_team.csv
    ├── train_proc.csv
    └── train_team.csv

1 directory, 13 files


In [3]:
# aditional
abilities = pd.read_csv( os.path.join(input_dir, 'dota2_abilities.csv') )
items = pd.read_csv( os.path.join(input_dir, 'dota2_items.csv') )
heroes = pd.read_csv( os.path.join(input_dir, 'dota2_heroes.csv') )
# Main dataset
train = pd.read_csv( os.path.join(input_dir, 'dota2_skill_train.csv'), index_col=0 )
test = pd.read_csv( os.path.join(input_dir, 'dota2_skill_test.csv'), index_col=0 )

new_features_train = pd.read_csv( os.path.join(processed_dir, 'train_proc.csv'), index_col=0 )
new_features_test = pd.read_csv( os.path.join(processed_dir, 'test_proc.csv'), index_col=0 )

team_train = pd.read_csv( os.path.join(processed_dir, 'train_team.csv'), index_col=0 )
team_test = pd.read_csv( os.path.join(processed_dir, 'test_team.csv'), index_col=0 )

In [4]:
print("Deleted", set(train.columns).difference( set(test.columns) ) )

y_train = train.skilled
train.drop(["skilled"], 1, inplace = True)

Deleted {'skilled'}


In [5]:
# Transformation
def apply(func, columns):
    global train, test
    for c in tqdm(columns):
        train[c] = train[c].apply(func)
        test[c] = test[c].apply(func)


def tr(x):
    if x=='dire':
        return 0.0
    elif x=='radiant':
        return 1.0
apply(tr, ["player_team", "winner_team"])

100%|██████████| 2/2 [00:00<00:00, 33.28it/s]


In [6]:
# New features
train = pd.concat((train, new_features_train, team_train), axis=1)
test = pd.concat((test, new_features_test, team_test), axis=1)

ntrain = train.shape[0]
ntest = test.shape[0]
all_data = pd.concat((train, test), axis=0)

In [7]:
all_data['_win'] = all_data['player_team'] == all_data['winner_team']
all_data['_kda'] = (all_data.kills + all_data.assists)/ all_data.deaths
all_data['_duration_min'] = all_data.duration / 60
all_data['__gold_dame'] = all_data['_duration_min'] * all_data.gold_per_min
all_data['__xp_dame'] = all_data['_duration_min'] * all_data.xp_per_min
all_data['__free_gold'] = all_data['__gold_dame'] - all_data.gold_spent

In [8]:
all_data["_radiant_gold_pre_sec"] = all_data["radiant_final_gold"] / all_data.duration
all_data["_dire_gold_pre_sec"] = all_data["dire_final_gold"] / all_data.duration
# add difs
all_data["__team_gold_per_sec_diff"] =  (all_data["_radiant_gold_pre_sec"] - all_data["_dire_gold_pre_sec"])\
                                                                        * (all_data.player_team - 0.5) * 2
all_data["_team_gold_diff"] = (all_data["radiant_final_gold"] - all_data["dire_final_gold"])\
                                                                        * (all_data.player_team - 0.5) * 2

In [9]:
all_data["radiant_win_%"] = abs( all_data["radiant_win_%"] - (1.0 * all_data.player_team) )

In [10]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [22]:
from sklearn.model_selection import cross_val_score, cross_val_predict
import catboost as cb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

In [12]:
cross_val_score(
    lgb.LGBMClassifier(),
    train.astype('float64').replace({np.inf:np.nan}).fillna(0),
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=1,
    n_jobs=-1
)

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

array([0.76546918, 0.76642102, 0.76777346, 0.76585092])

In [13]:
model = lgb.LGBMClassifier().fit(train, y_train)

In [17]:
y_pred = model.predict(test)
y_pred = pd.DataFrame({'id' : test.index, 'skilled': y_pred})
submit(y_pred, "raw_lgb")

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


File saved in : /home/furfa/work/ai-academy2019/predictions/raw_lgb[2019-03-27 02:05:19.776465].csv


In [14]:
params = {'boosting_type': ['gbdt'],
 'colsample_bytree': [0.64, 0.65, 0.66],
 'learning_rate': [0.03,0.001,0.01,0.1,0.2,0.3],
'num_iterations' : [250,100,500,1000],
 'max_bin': [512],
 'max_depth': [3,1,2,6,4,5,7,8,9,10],
 'metric': ['binary_error'],
 'min_child_samples': [5],
 'min_child_weight': [1],
 'min_split_gain': [0.5],
 'n_estimators': range(10,800,80),
 'nthread': [5],
 'num_class': [1],
 'num_leaves': [6, 8, 12, 16],
 'objective': ['binary'],
 'random_state': [501],
 'reg_alpha': [1, 1.2,1.3,1.4],
 'reg_lambda': [1, 1.2, 1.4,1.5,1.6],
 'scale_pos_weight': [1],
 'subsample': [0.7, 0.75],
 'subsample_for_bin': [200],
 'subsample_freq': [1]}
rs_lgb_model = RandomizedSearchCV(
                   lgb.LGBMClassifier(),
                   params,
                   n_iter=40,
                   n_jobs=-1)
rs_lgb_model.fit(train, y_train)

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0),
          fit_params=None, iid=True, n_iter=40, n_jobs=-1,
          param_distributions={'boosting_type': ['gbdt'], 'colsample_bytree': [0.64, 0.65, 0.66], 'learning_rate': [0.03, 0.001, 0.01, 0.1, 0.2, 0.3], 'num_iterations': [250, 100, 500, 1000], 'max_bin': [512], 'max_depth': [3, 1, 2, 6, 4, 5, 7, 8, 9, 10], 'metric': ['binary_error'], 'min_child_samples': [5], ...scale_pos_weight': [1], 'subsample': [0.7, 0.75], 'subsample_for_bin': [200], 'subsample_freq': [1]},
          pre_dispatch='2*n_jobs', random_state=None, ref

In [15]:
rs_lgb_model.best_score_

0.7877061409217891

In [16]:
grid_scores = rs_lgb_model.grid_scores_
grid_scores.sort(key=lambda x:x[1], reverse=True)

lgb_models = {"lgb "+str(i[1]):i[0] for i in grid_scores}

pd.DataFrame(
    lgb_models
)

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


,lgb 0.7877061409217891,lgb 0.7853531055061028,lgb 0.7840514263399786,lgb 0.7833204834236165,lgb 0.7829800442570917,lgb 0.7793253296752811,lgb 0.7779034955092069,lgb 0.7757907700934206,lgb 0.7745992330105836,lgb 0.7736880575942966,lgb 0.7723963913448348,lgb 0.7713750738452604,lgb 0.7708443892621482,lgb 0.7696828909292988,lgb 0.7616525317659781,lgb 0.7583783080173424,lgb 0.7537122888526199,lgb 0.743348920106938,lgb 0.7363398784431917,lgb 0.7361496330266043,lgb 0.7336564167776431,lgb 0.7320643630283065,lgb 0.7231128155320363,lgb 0.7183266413673639,lgb 0.7152426630353156,lgb 0.7135805188693415,lgb 0.7086441509547317,lgb 0.7072623684553073,lgb 0.7059707022058456,lgb 0.691021417628741,lgb 0.680107338466622,lgb 0.678204884300748,lgb 0.668932923471278,lgb 0.6582591543090587,lgb 0.6581690380590962,lgb 0.6512300868119875,lgb 0.6485165863964514,lgb 0.6016260976659892
boosting_type,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt
colsample_bytree,0.65,0.66,0.65,0.66,0.65,0.66,0.64,0.65,0.64,0.66,0.65,0.64,0.65,0.64,0.66,0.64,0.64,0.66,0.64,0.66,0.66,0.66,0.64,0.64,0.66,0.65,0.64,0.65,0.66,0.65,0.65,0.64,0.64,0.64,0.65,0.65,0.65,0.64
learning_rate,0.1,0.3,0.2,0.1,0.2,0.3,0.3,0.03,0.3,0.1,0.1,0.1,0.2,0.3,0.03,0.1,0.03,0.2,0.3,0.3,0.03,0.03,0.03,0.01,0.01,0.03,0.03,0.03,0.03,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.001,0.001
max_bin,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512
max_depth,4,2,10,9,5,5,6,8,8,8,3,10,2,8,6,2,6,1,1,1,6,5,3,7,4,1,7,6,5,1,8,1,1,1,1,4,9,2
metric,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error,binary_error
min_child_samples,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
min_child_weight,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
min_split_gain,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
n_estimators,170,490,410,250,90,90,570,570,570,490,330,490,90,330,410,410,250,490,250,170,730,90,570,90,410,410,410,730,650,730,490,10,730,90,250,650,490,90


In [23]:
cross_val_predict?

In [24]:
predicts_per_model = dict()
for model_name in tqdm_notebook( lgb_models.keys() ):
    params = lgb_models[model_name]
    model = lgb.LGBMClassifier(**params)
    predicts_per_model[model_name] = cross_val_predict(
        model, train, y_train,
        cv = 4,
        n_jobs=-1,
    )

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

In [38]:
# ищем наимение скореллированные модели
pd.DataFrame(predicts_per_model).corr().ix[:3]

,lgb 0.7877061409217891,lgb 0.7853531055061028,lgb 0.7840514263399786,lgb 0.7833204834236165,lgb 0.7829800442570917,lgb 0.7793253296752811,lgb 0.7779034955092069,lgb 0.7757907700934206,lgb 0.7745992330105836,lgb 0.7736880575942966,lgb 0.7723963913448348,lgb 0.7713750738452604,lgb 0.7708443892621482,lgb 0.7696828909292988,lgb 0.7616525317659781,lgb 0.7583783080173424,lgb 0.7537122888526199,lgb 0.743348920106938,lgb 0.7363398784431917,lgb 0.7361496330266043,lgb 0.7336564167776431,lgb 0.7320643630283065,lgb 0.7231128155320363,lgb 0.7183266413673639,lgb 0.7152426630353156,lgb 0.7135805188693415,lgb 0.7086441509547317,lgb 0.7072623684553073,lgb 0.7059707022058456,lgb 0.691021417628741,lgb 0.680107338466622,lgb 0.678204884300748,lgb 0.668932923471278,lgb 0.6582591543090587,lgb 0.6581690380590962,lgb 0.6512300868119875,lgb 0.6485165863964514,lgb 0.6016260976659892
lgb 0.7877061409217891,1.000000,0.863389,0.852069,0.878481,0.862165,0.821753,0.838767,0.850857,0.828933,0.839797,0.837090,0.824487,0.826519,0.808627,0.778550,0.769920,0.743283,0.695326,0.666175,0.664943,0.648970,0.645928,0.605306,0.585084,0.571261,0.565668,0.548034,0.542710,0.539109,0.482785,0.438126,0.428738,0.393609,0.354608,0.354603,0.336107,0.324307,NaN
lgb 0.7853531055061028,0.863389,1.000000,0.833763,0.862670,0.848704,0.806967,0.824227,0.827215,0.819869,0.816307,0.820721,0.804347,0.823365,0.792142,0.759758,0.761505,0.729551,0.693743,0.663794,0.663675,0.639305,0.635499,0.596698,0.576178,0.563046,0.563660,0.538701,0.533253,0.529377,0.479833,0.432135,0.425436,0.391031,0.350763,0.350532,0.327810,0.317324,NaN
lgb 0.7840514263399786,0.852069,0.833763,1.000000,0.844012,0.838384,0.801161,0.818630,0.825921,0.810583,0.817627,0.811317,0.802454,0.803076,0.794928,0.762666,0.752593,0.730605,0.683502,0.656159,0.655370,0.641291,0.637126,0.598848,0.579025,0.565138,0.559782,0.543304,0.537636,0.533981,0.478275,0.435397,0.425111,0.390765,0.351706,0.351530,0.333136,0.321955,NaN


In [43]:
# Т.к они не сильно коррелируют, то можно их стекнуть
from mlxtend.classifier import EnsembleVoteClassifier
voting = EnsembleVoteClassifier(
    clfs=[lgb.LGBMClassifier(**lgb_models[model_name]) for model_name in ("lgb 0.7877061409217891", 
                                                                        "lgb 0.7853531055061028",
                                                                        "lgb 0.7840514263399786")],
    voting="soft",
    verbose=1
)
cross_val_score(
    voting,
    train,
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=1,
    n_jobs=-1
)

Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf3: lgbmclassifier (3/3)


/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

array([0.79174176, 0.7925745 , 0.78916169, 0.78904153])

In [44]:
voting.fit(train, y_train)
y_pred = voting.predict(test)
y_pred = pd.DataFrame({'id' : test.index, 'skilled': y_pred})
submit(y_pred, "voting_non_correlated_soft")

Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf3: lgbmclassifier (3/3)
File saved in : /home/furfa/work/ai-academy2019/predictions/voting_non_correlated_soft[2019-03-28 16:35:34.582084].csv


/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [46]:
train.shape

(99871, 80)

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector
sfs = SequentialFeatureSelector(
    voting,
    k_features=40, 
    forward=False, 
    floating=True, 
    scoring='accuracy',
    cv=4,
    n_jobs=-1,
    verbose = 1,
)
sfs.fit(train.values, y_train.values)

Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf3: lgbmclassifier (3/3)


/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf3: lgbmclassifier (3/3)


/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf3: lgbmclassifier (3/3)


...

Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf2: lgbmclassifier (2/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf3: lgbmclassifier (3/3)
Fitting clf3: lgbmclassifier (3/3)


/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Fitting clf3: lgbmclassifier (3/3)


/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=-1)]: Done   4 out of  11 | elapsed:  5.4min remaining:  9.5min
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/furfa/anaconda3/lib/python3.6/site-packages/sklearn/preproces

Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting 3 classifiers...
Fitting clf1: lgbmclassifier (1/3)
Fitting clf2: lgbmclassifier (2/3)
Fitti

In [48]:
sfs.k_score_

### Как выяснилось catboost говнище

In [15]:
cb_grid = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
          'thread_count':[16],
          }
rs_cb_model = RandomizedSearchCV(
                   cb.CatBoostClassifier(logging_level="Silent"),
                   cb_grid,
                   n_iter=20,
                   cv=4,
                   verbose=2,
                   n_jobs=-1)
rs_cb_model.fit(train, y_train)

Fitting 4 folds for each of 20 candidates, totalling 80 fits
[CV] thread_count=16, learning_rate=0.03, l2_leaf_reg=3, iterations=500, depth=4, border_count=200 
[CV] thread_count=16, learning_rate=0.03, l2_leaf_reg=3, iterations=500, depth=4, border_count=200 
[CV] thread_count=16, learning_rate=0.03, l2_leaf_reg=3, iterations=500, depth=4, border_count=200 
[CV] thread_count=16, learning_rate=0.03, l2_leaf_reg=3, iterations=500, depth=4, border_count=200 
[CV] thread_count=16, learning_rate=0.3, l2_leaf_reg=100, iterations=500, depth=6, border_count=5 
[CV] thread_count=16, learning_rate=0.3, l2_leaf_reg=100, iterations=500, depth=6, border_count=5 
[CV] thread_count=16, learning_rate=0.3, l2_leaf_reg=100, iterations=500, depth=6, border_count=5 
[CV] thread_count=16, learning_rate=0.3, l2_leaf_reg=100, iterations=500, depth=6, border_count=5 
[CV] thread_count=16, learning_rate=0.1, l2_leaf_reg=3, iterations=1000, depth=5, border_count=5 
[CV] thread_count=16, learning_rate=0.1, l2_l

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.4min


[CV]  thread_count=16, learning_rate=0.3, l2_leaf_reg=100, iterations=500, depth=6, border_count=5, total= 3.3min
[CV]  thread_count=16, learning_rate=0.3, l2_leaf_reg=100, iterations=500, depth=6, border_count=5, total= 3.3min
[CV] thread_count=16, learning_rate=0.01, l2_leaf_reg=1, iterations=1000, depth=8, border_count=50 
[CV]  thread_count=16, learning_rate=0.3, l2_leaf_reg=100, iterations=500, depth=6, border_count=5, total= 3.3min
[CV] thread_count=16, learning_rate=0.01, l2_leaf_reg=1, iterations=1000, depth=8, border_count=50 
[CV] thread_count=16, learning_rate=0.01, l2_leaf_reg=1, iterations=1000, depth=8, border_count=50 
[CV]  thread_count=16, learning_rate=0.01, l2_leaf_reg=3, iterations=500, depth=3, border_count=32, total= 2.2min
[CV] thread_count=16, learning_rate=0.3, l2_leaf_reg=3, iterations=100, depth=7, border_count=200 
[CV]  thread_count=16, learning_rate=0.01, l2_leaf_reg=3, iterations=500, depth=3, border_count=32, total= 2.3min
[CV] thread_count=16, learning_

[CV] thread_count=16, learning_rate=0.1, l2_leaf_reg=3, iterations=100, depth=5, border_count=32 
[CV]  thread_count=16, learning_rate=0.03, l2_leaf_reg=100, iterations=100, depth=9, border_count=50, total= 1.3min
[CV] thread_count=16, learning_rate=0.1, l2_leaf_reg=5, iterations=100, depth=10, border_count=50 
[CV]  thread_count=16, learning_rate=0.03, l2_leaf_reg=100, iterations=100, depth=9, border_count=50, total= 1.2min
[CV] thread_count=16, learning_rate=0.1, l2_leaf_reg=5, iterations=100, depth=10, border_count=50 
[CV]  thread_count=16, learning_rate=0.03, l2_leaf_reg=100, iterations=100, depth=9, border_count=50, total= 1.3min
[CV] thread_count=16, learning_rate=0.1, l2_leaf_reg=5, iterations=100, depth=10, border_count=50 
[CV]  thread_count=16, learning_rate=0.03, l2_leaf_reg=100, iterations=100, depth=9, border_count=50, total= 1.3min
[CV] thread_count=16, learning_rate=0.1, l2_leaf_reg=5, iterations=100, depth=10, border_count=50 
[CV]  thread_count=16, learning_rate=0.1, 

[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 15.8min finished


RandomizedSearchCV(cv=4, error_score='raise',
          estimator=<catboost.core.CatBoostClassifier object at 0x7f58f8e3f358>,
          fit_params=None, iid=True, n_iter=20, n_jobs=-1,
          param_distributions={'depth': [3, 1, 2, 6, 4, 5, 7, 8, 9, 10], 'iterations': [250, 100, 500, 1000], 'learning_rate': [0.03, 0.001, 0.01, 0.1, 0.2, 0.3], 'l2_leaf_reg': [3, 1, 5, 10, 100], 'border_count': [32, 5, 10, 20, 50, 100, 200], 'thread_count': [16]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [20]:
rs_cb_model.best_score_

0.7771224880095323

In [21]:
y_pred = rs_cb_model.best_estimator_.predict(test)
y_pred = pd.DataFrame({'id' : test.index, 'skilled': y_pred})
submit(y_pred, "cb_grid")

File saved in : /home/furfa/work/ai-academy2019/predictions/cb_grid[2019-03-27 16:49:23.434588].csv
